## Welcome to our ETL Project

In [36]:
# Dependencies and Setup
import pandas as pd
import scipy.stats as st
import datetime
from datetime import date
from sqlalchemy import create_engine

## Import Earthquake Dataset

In [37]:
#Read .CSV
turkeyEQ = pd.read_csv("Data/Turkey_EQ.csv")
japanEQ = pd.read_csv("Data/Japan1_EQ.csv")

## Examine Turkey Earthquake Dataset

In [38]:
#Examine data
turkeyEQ.head(3)

,No,deprem_kodu,olus_tarihi,olus_zamani,enlem,boylam,derinlik,xM,MD,ML,Mw,Ms,Mb,Tip,Yer
0,16417,1.900090e+13,1900.09.20,00:01.0,37.8,29.1,5.0,5.0,5.0,0.0,NaN,0.0,0.0,Ke,DENIZLI (DENIZLI) [North East 2.3 km]
1,16416,1.901020e+13,1901.02.23,00:00.0,37.9,27.9,15.0,4.8,4.7,4.6,4.8,4.6,4.7,Ke,KENGER- (AYDIN) [North East 1.1 km]
2,16415,1.901030e+13,1901.03.01,00:01.0,38.2,27.7,5.0,5.0,5.0,0.0,NaN,0.0,0.0,Ke,YAKACIK-BAYINDIR (IZMIR) [South West 0.8 km]


In [39]:
#Rename translate to english
turkey_df=turkeyEQ.rename(columns={'No':'no', 'deprem_kodu': 'code', 'olus_tarihi': 'date', 'olus_zamani':'time', 'enlem':'latitude'
         ,'derinlik':'longitude','boylam':'depth','xM':'mag','MD':'MD', 'ML':'ML', 'Mw':'Mw', 'Ms':'Ms','Mb':'Mb','Tip':'type' , 'Yer':'place'})

turkey_df['date'] = pd.to_datetime(turkey_df.date)
turkey_df.head(2)

,no,code,date,time,latitude,depth,longitude,mag,MD,ML,Mw,Ms,Mb,type,place
0,16417,1.900090e+13,1900-09-20,00:01.0,37.8,29.1,5.0,5.0,5.0,0.0,NaN,0.0,0.0,Ke,DENIZLI (DENIZLI) [North East 2.3 km]
1,16416,1.901020e+13,1901-02-23,00:00.0,37.9,27.9,15.0,4.8,4.7,4.6,4.8,4.6,4.7,Ke,KENGER- (AYDIN) [North East 1.1 km]


In [40]:
Turkeycleaned_df = turkey_df[["date", "latitude","longitude", "mag", "depth","place"]]                            
Turkeycleaned_df.head()                          

,date,latitude,longitude,mag,depth,place
0,1900-09-20,37.8,5.0,5.0,29.1,DENIZLI (DENIZLI) [North East 2.3 km]
1,1901-02-23,37.9,15.0,4.8,27.9,KENGER- (AYDIN) [North East 1.1 km]
2,1901-03-01,38.2,5.0,5.0,27.7,YAKACIK-BAYINDIR (IZMIR) [South West 0.8 km]
3,1901-04-01,38.4,5.0,5.0,31.4,ATAKENT-AKSEHIR (KONYA) [North East 2.4 km]
4,1901-05-01,37.8,15.0,5.0,27.8,KADIKOY- (AYDIN) [South West 2.6 km]


In [41]:
#dtypes
Turkeycleaned_df.dtypes

date         datetime64[ns]
latitude            float64
longitude           float64
mag                 float64
depth               float64
place                object
dtype: object

## Examine Japan Earthquake Dataset

In [42]:
#Examine data
Japan_df = pd.DataFrame(japanEQ)
Japan_df['date'] = pd.to_datetime(Japan_df.date)
Japan_df.head(2)

,date,Time,latitude,longitude,depth,mag,magType,nst,gap,dmin,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2018-11-27,T14:34:20.900Z,48.3780,154.962,35.00,4.9,mb,NaN,92.0,5.044,...,2018-11-27T16:06:33.040Z,"269km SSW of Severo-Kuril'sk, Russia",earthquake,7.6,1.7,0.036,248.0,reviewed,us,us
1,2018-11-26,T23:33:50.630Z,36.0733,139.783,48.82,4.8,mww,NaN,113.0,1.359,...,2018-11-27T16:44:22.223Z,"3km SSW of Sakai, Japan",earthquake,6.0,6.1,0.071,19.0,reviewed,us,us


In [43]:
Japancleaned_df = Japan_df[["date", "latitude","longitude", "mag", "depth","place"]]                            
Japancleaned_df.head() 

,date,latitude,longitude,mag,depth,place
0,2018-11-27,48.3780,154.9620,4.9,35.00,"269km SSW of Severo-Kuril'sk, Russia"
1,2018-11-26,36.0733,139.7830,4.8,48.82,"3km SSW of Sakai, Japan"
2,2018-11-26,38.8576,141.8384,4.5,50.56,"26km SSE of Ofunato, Japan"
3,2018-11-26,50.0727,156.1420,4.6,66.34,"67km S of Severo-Kuril'sk, Russia"
4,2018-11-25,33.9500,134.4942,4.6,38.19,"9km SW of Komatsushima, Japan"


In [44]:
#check datatypes
Japancleaned_df.dtypes

date         datetime64[ns]
latitude            float64
longitude           float64
mag                 float64
depth               float64
place                object
dtype: object

## join both datasets and transfer to SQL

In [50]:
#Join both datasets
earthquake_db = pd.concat([Turkeycleaned_df, Japancleaned_df], axis=0, join='outer', ignore_index=False)

earthquake_db

,date,latitude,longitude,mag,depth,place
0,1900-09-20,37.800,5.000,5.0,29.1,DENIZLI (DENIZLI) [North East 2.3 km]
1,1901-02-23,37.900,15.000,4.8,27.9,KENGER- (AYDIN) [North East 1.1 km]
2,1901-03-01,38.200,5.000,5.0,27.7,YAKACIK-BAYINDIR (IZMIR) [South West 0.8 km]
3,1901-04-01,38.400,5.000,5.0,31.4,ATAKENT-AKSEHIR (KONYA) [North East 2.4 km]
4,1901-05-01,37.800,15.000,5.0,27.8,KADIKOY- (AYDIN) [South West 2.6 km]
...,...,...,...,...,...,...
14087,2001-01-04,36.981,138.629,5.3,21.4,"eastern Honshu, Japan"
14088,2001-01-03,43.932,147.813,5.9,33.0,Kuril Islands
14089,2001-01-03,45.301,149.941,4.6,33.0,Kuril Islands
14090,2001-01-02,32.239,141.508,4.9,33.0,"Izu Islands, Japan region"


In [51]:
#save dataframe to to .csv
earthquake_db.to_csv("Data/earthquake.csv", index = False, header=True)

In [53]:
#Read .csv to sql
pd.read_csv("Data/earthquake.csv").to_sql(name="earthquake", con=create_engine("sqlite:///my_db.sqlite"))

## Project complete please see our website and project documents in github